# Event Extraction (SVO)
#### parameters : news(.pkl) / output_path(.pkl)

In [3]:
# input
import pickle
with open('./dataset/test.pkl', 'rb') as f:
    data = pickle.load(f)
data.head()

,id,title,date,source,document_body
0,0,Railcars in North Dakota crude train crash old...,20140101,Reuters News,* Railcars don't meet industry's latest safety...
1,1,S.Korea Dec crude oil imports down 8.4 pct y/y...,20140101,Reuters News,"SEOUL, Jan 1 (Reuters) - South Korea's crude o..."
2,2,"Doctors, hospitals expect some confusion as Ob...",20140101,Reuters News,Jan 1 (Reuters) - Hospitals and medical practi...
3,3,Justice Blocks Contraception Mandate on Insura...,20140101,The New York Times,WASHINGTON -- Justice Sonia Sotomayor on Tuesd...
4,4,"For Stocks, an Amazingly Good Year",20140101,The New York Times,It was the market rally that defied gravity an...


In [3]:
!python3 Event_Extraction.py ./dataset/test.pkl ./dataset/integrate_SVO.pkl

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 18.21it/s]


In [3]:
# output
with open('./dataset/integrate_SVO.pkl', 'rb') as f:
    data = pickle.load(f)
data[0]

{'id': 0,
 'date': '20140101',
 'title': 'Railcars in North Dakota crude train crash older, less safe -investigators',
 'title_SVO': defaultdict(list,
             {'main': [{'subject': [('Railcars',
                  [defaultdict(list,
                               {'predicate': [('in', [])],
                                'object': [('North Dakota crude train crash',
                                  [])]})])],
                'predicate': [],
                'object': []}]}),
 'integrate_SVO': [[('Railcars', ''),
   ('in', ''),
   ('North Dakota crude train crash', '')]]}

# Convert to Token (Word_Embedding)
#### parameters : integrate_SVO(.pkl) / output_path(.pkl)

In [4]:
!python3 Word_Embedding.py ./dataset/integrate_SVO.pkl ./dataset/news_embedding.pkl

100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 5841.65it/s]
0it [00:00, ?it/s]                                        | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 2/2 [00:02<00:00,  1.35s/it]


In [5]:
# output
# Each Event: tensor(3, 768)
with open('./dataset/news_embedding.pkl', 'rb') as f:
    data = pickle.load(f)
data[0]

{'date': '20140101',
 'SVO': [tensor([[ 0.1677,  0.5718,  0.4815,  ..., -0.0256,  0.6719,  0.4798],
          [ 0.3100,  0.7907,  0.0308,  ...,  0.0147,  0.3137, -0.0888],
          [-0.4872,  0.1933,  0.1532,  ...,  0.0936,  0.2514,  0.2345]]),
  tensor([[-0.3281,  0.1918,  0.2351,  ...,  0.2562,  0.4863,  0.1667],
          [-0.2758, -0.2907,  0.3800,  ..., -0.0280,  0.5327, -0.3334],
          [-0.4366, -0.0357,  0.0170,  ...,  0.3978,  0.2332,  0.1756]]),
  tensor([[-0.2246,  0.3442,  0.3954,  ...,  0.0136,  0.6432,  0.4880],
          [-0.7752, -0.5799,  0.0325,  ...,  0.0741, -0.3688,  0.1206],
          [-0.0034,  0.1318, -0.0187,  ..., -0.0985,  0.8148, -0.0674]]),
  tensor([[ 0.2384,  0.5718,  0.6301,  ...,  0.4288,  0.2098,  0.3832],
          [-0.3787, -0.0049, -0.3886,  ..., -0.1581,  0.3159, -0.0910],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([[-0.4717,  0.1754,  0.1051,  ..., -0.0825,  0.1905, -0.0989],
          [ 0.2616,  0.6423,

# Event Embedding (AutoEncoder)
#### parameters: news_embedding(.pkl) / Autoencoder model(.pth)  / outputpath(.pkl)

In [1]:
!python3 Event_Embedding.py ./dataset/news_embedding.pkl autoencoder_model.pth ./dataset/event_embedding.pkl

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  3.36it/s]


In [6]:
# output
# dictionary - key: date / value: array(768)
with open('./dataset/event_embedding.pkl', 'rb') as f:
    data = pickle.load(f)
data.keys()

dict_keys(['20140101', '20140102'])

# AutoEncoder Training
#### parameters: news_embedding(.pkl)

In [1]:
!python3 AutoEncoder_train.py ./dataset/news_embedding.pkl

1it [00:01,  1.11s/it]
Epoch 000: Loss : 0.50690
model saved to autoencoder_1.pth


# Stock Prediction (LSTM)
#### parameters: price(.csv) / event_embedding(.pkl) / LSTM model(.pth) / output_path(.csv)

In [3]:
!python3 LSTM_test.py ./dataset/price.csv ./dataset/Event_embedding.pkl './model/LSTM(3day_test).pth' ./result/test.csv

100%|███████████████████████████████████████| 140/140 [00:00<00:00, 1949.46it/s]
acc:  0.6928571428571428
F1:  0.6323092677931388
precision 0.6988095238095239
recall 0.6928571428571428


In [4]:
import pandas as pd
data = pd.read_csv('./result/test.csv')
data.head()

,date,true,predict
0,20190506,1,0
1,20190507,0,0
2,20190508,0,0
3,20190509,0,0
4,20190510,0,0


# LSTM Training
#### parameters: price(.csv) / event_embedding(.pkl)

In [1]:
!python3 LSTM_train.py ./dataset/price.csv ./dataset/Event_embedding.pkl

100%|████████████████████████████████████████| 164/164 [00:00<00:00, 635.63it/s]
Epoch: 1, train Loss: 0.9098, train accuracy: 0.4069
Epoch: 1, valid loss: 0.7837, valid accuracy: 0.1875
Epoch: 2, train Loss: 0.7055, train accuracy: 0.6033
Epoch: 2, valid loss: 0.8202, valid accuracy: 0.1875


# Market Simulation
#### parameters: price(.csv) / predict_ans(.csv)

In [5]:
!python3 Market_Simulation.py ./dataset/price.csv ./result/test.csv

Return:  1.1105337761774103


# Baseline Word Embedding
#### parameters: news(.pkl) / output_path(.pkl)

In [1]:
!python3 Baseline_preprocess.py ./dataset/test.pkl ./dataset/baseline_test.pkl

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 14217.98it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]                                        | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.27it/s]


In [3]:
# output
# dictionary - key: date / value: array(768)
with open('./dataset/baseline_test.pkl', 'rb') as f:
    data = pickle.load(f)
data.keys()

dict_keys(['20140102', '20140101'])

# RTF to Dataframe
#### parameters: list(input_path) / output_path(.pkl)

In [ ]:
paths = []
!python3 RTF_to_Dataframe.py paths ./dataset/news.pklb